In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils
import os
import numpy as np
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D,Convolution2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.optimizers import Adam

import pandas as pd

In [ ]:

NUM_CHANNELS = 10
IMAGE_WIDTH = 224 # Original: 455
IMAGE_HEIGHT = 224 # Original: 256
NUM_CLASSES = 10

model = Sequential([
Convolution2D(16, 10, 10, border_mode='same', subsample=(2, 2), input_shape=
(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CLASSES), activation='relu'),
MaxPooling2D(pool_size=(3, 3)),
Dropout(0.2),

Convolution2D(32, 3, 3, border_mode='same', activation='relu'),
MaxPooling2D(pool_size=(3, 3)),
Dropout(0.2),

Convolution2D(64, 3, 3, border_mode='same', activation='relu'),
MaxPooling2D(pool_size=(2, 2)),
Dropout(0.2),

Flatten(),
Dense(128, activation='tanh'),
Dropout(0.3),
Dense(NUM_CLASSES, activation='softmax'),
])
model.summary()

model.compile(optimizer=Adam(lr=0.0003), loss='categorical_crossentropy', 
metrics=['accuracy'])


In [ ]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
num_classes = len(np.unique(y_train))
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=10)

# here's a more "manual" example
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
        model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(x_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

In [3]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [4]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/20
1563/1563 [==============================] - 173s 111ms/step - loss: 1.8486 - accuracy: 0.3201 - val_loss: 1.5660 - val_accuracy: 0.4349
Epoch 2/20
1563/1563 [==============================] - 199s 127ms/step - loss: 1.5865 - accuracy: 0.4196 - val_loss: 1.4114 - val_accuracy: 0.4893
Epoch 3/20
1563/1563 [==============================] - 194s 124ms/step - loss: 1.4651 - accuracy: 0.4703 - val_loss: 1.3039 - val_accuracy: 0.5351
Epoch 4/20
1563/1563 [==============================] - 193s 124ms/step - loss: 1.3829 - accuracy: 0.5032 - val_loss: 1.3101 - val_accuracy: 0.5389
Epoch 5/20
1563/1563 [==============================] - 194s 124ms/step - loss: 1.3155 - accuracy: 0.5283 - val_loss: 1.2017 - val_accuracy: 0.5744
Epoch 6/20
1563/1563 [==============================] - 193s 123ms/step - loss: 1.2582 - accuracy: 0.5511 - val_loss: 1.1960 - val_accuracy: 0.5755
Epoc